In [ ]:
import torch
import torch.nn as nn
from oct_library import OCTProcessing
model_path = '../logs/2022-11-09_19_35_23/checkpoints/model.pth'
oc_file = '../dataset/vol/hc04_spectralis_macula_v1_s1_R.vol'
model = torch.load(model_path, map_location='cuda')
# model = model.half()


In [ ]:

import torch.onnx
dummy_input=torch.randn(32, 1, 256, 256, device='cuda')
torch.onnx.export(model, dummy_input, "unet_oct.onnx", verbose=True)


In [ ]:
import tensorrt
!trtexec --onnx=unet_oct.onnx --saveEngine=unet_engine_pytorch.trt  --explicitBatch --inputIOFormats=fp16:chw --outputIOFormats=fp16:chw --fp16

In [ ]:
import numpy as np
oct_process = OCTProcessing(oct_file=oc_file, torchmodel=model, half=False, device='cuda') # 125, 36, 10, 68, 15
oct_process.fovea_forward(imgh=256, imgw=256)
# tempCPU = []
# tempGPU = []
# pwCPU = []
# pwGPU = []
# fps = []

for i in range(100):
    oct_process.fovea_forward(imgh=256, imgw=256)
#     fps.append(oct_process.FPS)
#     tempCPU.append(temp_cpu)

# print('FPS', np.array(fps).mean(), np.array(fps).std())
# print('temp CPU', np.array(tempCPU).mean(), np.array(tempCPU).std())
# print('temp GPU',np.array(tempGPU).mean(), np.array(tempGPU).std())
# print('Power CPU',np.array(pwCPU).mean(), np.array(pwCPU).std())
# print('Power GPU',np.array(pwGPU).mean(), np.array(pwGPU).std())

In [ ]:
oct_process.plot_slo_fovea()


In [ ]:
oct_process.plot_overlay_oct_segmentation()
oct_process.plot_segmentation_full()
oct_process.get_individual_layers_segmentation(layer=[False, True, True, False, True, True, False, False, True])
oct_process.plot_selected_layers()